In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools

In [2]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [3]:
from typing import Union, List, Tuple

# Index

This second notebook covers the following intermediate Graph Theory topics:

1. [Multigraphs](#Multigraphs)
2. [Graph Representations and the Adjacency Matrix](#Graph-Representations-and-the-Adjacency-Matrix)

# Intermediate concepts in Graph Theory: from *Multigraphs* to *Bipartite Graphs*